<a href="https://colab.research.google.com/github/mattiadido95/Haze-Fog-suppression/blob/main/dataset_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [24]:
from google.colab import drive
from google.colab.patches import cv2_imshow
import os
import gdown
import zipfile
import cv2
import numpy as np

In [2]:
# mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
folder_path = "drive/MyDrive/Haze-Fog-suppression"
os.chdir(folder_path)

# Uncompress Zip files


In [15]:
#os.chdir("dataset")
current_directory = os.getcwd()

#estrae tutti i file zip e cancella il file compresso
for file_name in os.listdir(current_directory):
    if file_name.endswith(".zip"):
        path_to_zip_file = os.path.join(current_directory, file_name)
        with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
            zip_ref.extractall(current_directory)
        os.remove(path_to_zip_file)

# Generate datasets

In [16]:
zero_beta = 0
low_beta = 0
medium_beta = 0
high_beta = 0

path_to_original_image = "/content/drive/MyDrive/Haze-Fog-suppression/dataset/DepthMaps/00000.png"
path_to_depth_map = "/content/drive/MyDrive/Haze-Fog-suppression/dataset/Vis/00000.png"

!ls

DepthMaps  SOTS  Vis


In [43]:
# Carica l'immagine originale e la mappa di transizione
J = cv2.imread(path_to_original_image)
d = cv2.imread(path_to_depth_map)

# Specifica i parametri
A = 100  # Valore di luce atmosferica (generalmente tra 0 e 255)
beta = low_beta

#Calcola la mappa di transizione
t = np.exp(-beta * d)

print(J.shape)
print(t.shape)
print(d.shape)

# Calcola l'immagine degradata con effetto nebbia
I = J * t + A * (1 - t)

print(I.shape)
# Visualizza l'immagine degradata
#cv2.imshow('Degraded Image', I)
#cv2.waitKey(0)

# Salva l'immagine degradata su disco
cv2.imwrite('degraded_image.jpg', I)



(768, 1024, 3)
(768, 1024, 3)
(768, 1024, 3)
(768, 1024, 3)


error: ignored